# 第5章 决策树

配置环境：python 3.6

代码全部测试通过。

此文档方便阅读讲解，若需要复制粘贴可以在当前目录中查看`DecisionTree.py`


In [1]:
import numpy as np
import pandas as pd
from math import log

In [2]:
class DNode:
    def __init__(self, root=None, feature_name=None, label=None):
        """
        :param root root=False 是分类节点 root=True 是叶节点
        :param feature_name feature_name 表示分类节点的分类名字
        :param label 根节点的类别，如果还有类别不唯一，通常是分类到最后，最多的那个类别，如果不是叶节点
        label 为 None
        """
        
        self.root = root
        self.feature_name = feature_name
        self.label = label
        self.tree = {}
        self.result = {'label': self.label, 'feature_name': self.feature_name, 'tree': self.tree}
    
    def __repr__(self):
        """
        输出这个对象的时候调用这个函数
        """
        return '{}'.format(self.result)

    def add_node(self, val, node):
        """
        添加子节点
        """
        self.tree[val] = node

In [ ]:
class DTree:
    def __init__(self, epsilon=0.1):
        """
        epsilon 信息增益的阈值
        如果最大信息增益小于阈值，就不继续分节点了
        """
        self._tree = None
        self.epsilon = epsilon

    @staticmethod
    def cal_ent(dataset):
        """
        
        """
        data_length = len(dataset)
        labels = {}
        for i in range(data_length):
            label = dataset[i][-1]
            if label not in labels:
                labels[label] = 0
            labels[label] += 1
        ent = -sum([(p/data_length)*log(p/data_length, 2) 
                for p in labels.values()])
        print('ent', ent)
        
        return ent

    def cond_ent(self, dataset, axis=0):
        # 计算经验条件熵
        data_length = len(dataset)
        cla_via_feature = {}
        for r in range(data_length):
            f = dataset[r][axis]
            if f not in cla_via_feature:
                cla_via_feature[f] = []
            cla_via_feature[f].append(dataset[r])
        ent = sum([(len(p)/data_length)*self.cal_ent(p) 
                    for p in cla_via_feature.values()])
        return ent

    def info_gain(self, ent, c_ent):
        return ent - c_ent

    def info_gain_train(self, dataset):
        count = len(dataset[0]) - 1
        # 计算熵
        ent = self.cal_ent(dataset)
        best_feature = []
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(dataset, axis=c))
            best_feature.append((c, c_info_gain))
        best_ = max(best_feature, key=lambda x: x[-1])

        return best_

    def train(self, train_data):
        """
        递归函数
        1. 先得到数据
            数据格式是 pandas
        2. 判断是否是单节点树
            1. 所有分类的样本的标签是一样的
            2. 虽然有的标签不一样，但是特征值用完了，现在强行弄成标签一样，类别最多的那个
        3. 计算经验熵和条件经验熵，计算信息增益，算出信息增益最大的那个
            1. 如果最大信息增益小于阈值，强行弄成标签一样，类别最多的那个
        4. 把由 3 产生的数据重复执行前三步
        """ 
        y_train, features = train_data.iloc[:, -1], train_data.columns[:-1]
        
        # 如果是都是一样的类，返回单节点树
        if len(y_train.value_counts()) == 1:
            return DNode(root=True, label=y_train.iloc[0])
        
        # 没有可分的特征值了，返回单节点树
        if len(features) == 0:
            return DNode(root=True, 
                        label=y_train.value_counts().sort_value(ascending=False).index[0])
        
        # 计算经验熵和经验条件熵
        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]
        print('max_feature max_feature_name', max_feature, max_feature_name)

        if max_info_gain < self.epsilon:
            return DNode(root=True,
                        label=y_train.value_counts().sort_value(ascending=False).index[0])

        # 构建子集
        node_tree = DNode(root=False, 
                        feature_name=max_feature_name)

        # 将那些数据按 max_feature_name 分开
        feature_list = train_data[max_feature_name].value_counts().index

        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop(
                [max_feature_name], axis=1)

            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree
